In [8]:
import pandas as pd
import numpy as np
import os
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization, Bidirectional
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
# dir_path = './' #local
dir_path = './drive/MyDrive/BAKA/' #colab

In [9]:
#uncomment in colab
from google.colab import drive
drive.mount('/content/drive')
# pd.read_csv(dir_path + 'DataCollection' + '/' + '1' + '/' + '0.txt', sep=' ', index_col=False )
# test = np.genfromtxt(dir_path + 'DataCollection' + '/' + '1' + '/' + '0.txt', delimiter=' ',dtype='float64')
df = pd.read_csv(dir_path + 'DataCollection' + '/' + '1' + '/' + '0.txt', header=None, sep=' ')
# df.drop(df.index[60:])
# filler = np.zeros((100-len(df), 31))
# df = df.append(pd.DataFrame(filler), ignore_index=True )
# df
if len(df.columns) > 31:
  df = df.drop(columns=[31])
# df = df.drop(df.index[60:])
# df
# sp_df = np.array_split(df, 2)
# sp_df[0]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# for i in ['6']:
#     samples = os.listdir(dir_path + 'DataCollection' + '/' + i)
#     num_tests = int(len(samples)/5);
#     shuffle(samples, random_state = 0)
#     for k in range(0, len(samples)):
# #         df = np.genfromtxt(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], delimiter=' ',dtype='float64')
#         df = pd.read_csv(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], header=None, sep=' ')
#         df = df.drop(df.index[60:])
#         if len(df.columns) > 31:
#           df = df.drop(columns=[31])
#         if df.isnull().values.any():
#           print(i, " ", samples[k])
#         print(samples[k])
#         display(df)
#         # # if len(df) < 100:
#         # #     filler = np.zeros((100-len(df), 31))
#         # #     df = df.append(pd.DataFrame(filler), ignore_index=True )
#         # x_test.append(df.to_numpy())
#         # y_test.append(int(i));

In [10]:
files = os.listdir(dir_path + 'DataCollection')
files

['0', '2', '3', '4', '1', '5', '6']

In [11]:
x_train = [];
y_train = [];

x_test = [];
y_test = [];
for i in files:
    samples = os.listdir(dir_path + 'DataCollection' + '/' + i)
    num_tests = int(len(samples)/5);
    shuffle(samples, random_state = 0)
    for k in range(0, num_tests):
#         df = np.genfromtxt(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], delimiter=' ',dtype='float64')
        df = pd.read_csv(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], header=None, sep=' ')
        df = df.drop(df.index[60:])
        if len(df.columns) > 31:
          df = df.drop(columns=[31])
        if df.isnull().values.any():
          print(i, " ", samples[k])
        # if len(df) < 100:
        #     filler = np.zeros((100-len(df), 31))
        #     df = df.append(pd.DataFrame(filler), ignore_index=True )
        x_test.append(df.to_numpy())
        y_test.append(int(i));
    
    for k in range(num_tests, len(samples)):
        df = pd.read_csv(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], header=None, sep=' ')
        df = df.drop(df.index[60:])
        if len(df.columns) > 31:
          df = df.drop(columns=[31])
        if df.isnull().values.any():
          print(i, " ", samples[k])
        # df = df.drop(df.index[100:])
        # if len(df) < 100:
        #     filler = np.zeros((100-len(df), 31))
        #     df = df.append(pd.DataFrame(filler), ignore_index=True )
        x_train.append(df.to_numpy())
        y_train.append(int(i));
    
    print(len(samples), ' ', num_tests, ' ', len(samples)- num_tests)
# print(len(x_train))
x_train = np.array(x_train)
y_train = np.array(y_train);

x_test = np.array(x_test)
y_test = np.array(y_test)
print("x_train.shape: ", x_train.shape)
print("y_train.shiape: ", y_train.shape)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: ", y_test.shape)


302   60   242
512   102   410
302   60   242
410   82   328
602   120   482
422   84   338
417   83   334
x_train.shape:  (2376, 60, 31)
y_train.shiape:  (2376,)
x_test.shape:  (591, 60, 31)
y_test.shape:  (591,)


In [ ]:
print("x_train.shape: ", x_train.shape)
print("y_train.shiape: ", y_train.shape)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: ", y_test.shape)



x_train.shape:  (2376, 60, 31)
y_train.shiape:  (2376,)
x_test.shape:  (591, 60, 31)
y_test.shape:  (591,)


In [ ]:
def scale_data(data, min_max_scaler):
    for i in range(len(data)):
        data[i] = min_max_scaler.transform(data[i])
    return data

In [12]:
min_max_scaler = MinMaxScaler(feature_range=(0,1))

num_instances, num_time_steps, num_features = x_train.shape
x_train = np.reshape(x_train, newshape=(-1, num_features))
x_train = min_max_scaler.fit_transform(x_train)
x_train = np.reshape(x_train, newshape=(num_instances, num_time_steps, num_features))

x_train, y_train = shuffle(x_train, y_train, random_state=0)

num_instances, num_time_steps, num_features = x_test.shape
x_test = np.reshape(x_test, newshape=(-1, num_features))
x_test = min_max_scaler.transform(x_test)
x_test = np.reshape(x_test, newshape=(num_instances, num_time_steps, num_features))

x_test, y_test = shuffle(x_test, y_test, random_state=0)


In [6]:
# x_train = np.delete(x_train, [0,1,2,3,4,5,6,7,8], 0)
# y_train = np.delete(y_train, [0,1,2,3,4,5,6,7,8], 0)
print("x_train.shape: ", x_train.shape)
print("y_train.shiape: ", y_train.shape)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: ", y_test.shape)
# print(y_train)
# print("\n")
# print(y_test)

# y_train = y_train.astype('int')

# print(y_train)


x_train.shape:  (2376, 60, 31)
y_train.shiape:  (2376,)
x_test.shape:  (591, 60, 31)
y_test.shape:  (591,)


In [23]:
model = Sequential()
model.add(Bidirectional(LSTM(units=60, return_sequences=True ,dtype='float64'),input_shape=x_train.shape[1:],dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Bidirectional(LSTM(units=60 ,dtype='float64') ,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))


# model.add(Dense(30, activation='softmax'))

model.add(Dense(7, activation='softmax',dtype='float64'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_6 (Bidirection (None, 60, 120)           44160     
_________________________________________________________________
batch_normalization_6 (Batch (None, 60, 120)           480       
_________________________________________________________________
dropout_6 (Dropout)          (None, 60, 120)           0         
_________________________________________________________________
bidirectional_7 (Bidirection (None, 120)               86880     
_________________________________________________________________
batch_normalization_7 (Batch (None, 120)               480       
_________________________________________________________________
dropout_7 (Dropout)          (None, 120)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 7)                

In [24]:
# model = Sequential()
# model.add(LSTM(units=60, input_shape=x_train.shape[1:], return_sequences=True,dtype='float64'))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))

# model.add(LSTM(60, return_sequences=True ,dtype='float64'))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))

# model.add(LSTM(60, return_sequences=True ,dtype='float64'))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))

# model.add(LSTM(60,dtype='float64'))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))

# # model.add(Dense(30, activation='softmax'))

# model.add(Dense(7, activation='softmax',dtype='float64'))
# model.summary()

In [25]:
opt = tf.keras.optimizers.Adam(lr=0.0001, decay=1e-5)

checkpoint_filepath = dir_path + 'Checkpoints/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)


gestures = model.fit(x = x_train,
            y = y_train,
            epochs=50,
            validation_split=0.1, #split 10% of the trainning set for the validation set,
            batch_size=24,
            callbacks=[model_checkpoint_callback],
            shuffle=True
         )

Epoch 1/50
90/90 [==============================] - 15s 107ms/step - loss: 1.6631 - accuracy: 0.4298 - val_loss: 1.6215 - val_accuracy: 0.4622
Epoch 2/50
90/90 [==============================] - 8s 94ms/step - loss: 0.3995 - accuracy: 0.8731 - val_loss: 1.1969 - val_accuracy: 0.5504
Epoch 3/50
90/90 [==============================] - 8s 94ms/step - loss: 0.2547 - accuracy: 0.9198 - val_loss: 0.6698 - val_accuracy: 0.7521
Epoch 4/50
90/90 [==============================] - 8s 92ms/step - loss: 0.2165 - accuracy: 0.9284 - val_loss: 0.4043 - val_accuracy: 0.8487
Epoch 5/50
90/90 [==============================] - 8s 92ms/step - loss: 0.2009 - accuracy: 0.9468 - val_loss: 0.1552 - val_accuracy: 0.9622
Epoch 6/50
90/90 [==============================] - 8s 94ms/step - loss: 0.1429 - accuracy: 0.9503 - val_loss: 0.0806 - val_accuracy: 0.9790
Epoch 7/50
90/90 [==============================] - 8s 93ms/step - loss: 0.1455 - accuracy: 0.9595 - val_loss: 0.0527 - val_accuracy: 0.9874
Epoch 8/50


In [26]:
# model.save(dir_path + 'Models/gestures.h5')

In [27]:
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=24)
print("test loss, test acc:", results)
model.save(dir_path + 'Models/gestures_bidir', save_format='tf')

Evaluate on test data
25/25 [==============================] - 1s 30ms/step - loss: 0.0496 - accuracy: 0.9814
test loss, test acc: [0.049636900424957275, 0.9813874959945679]


INFO:tensorflow:Assets written to: ./drive/MyDrive/BAKA/Models/gestures_bidir/assets


INFO:tensorflow:Assets written to: ./drive/MyDrive/BAKA/Models/gestures_bidir/assets


In [28]:
# new_model = load_model(dir_path + 'Models/gestures.h5')
# print("Evaluate on test data")
# results = new_model.evaluate(x_test, y_test, batch_size=80)
# print("test loss, test acc:", results)

In [29]:
min_max_scaler.data_min_

array([ 9.50955e+01,  8.99999e+01,  9.00456e+01,  9.00456e+01,
        9.00455e+01,  6.54812e+01,  2.57840e+01,  3.11686e+01,
        2.83181e+01,  2.04904e+01, -4.65014e+02,  6.15573e+01,
       -2.90505e+02, -4.80121e+02,  3.30287e+01, -3.15328e+02,
       -4.71490e+02,  2.19222e+01, -3.17749e+02, -4.59829e+02,
        3.47366e+01, -3.13954e+02, -4.57886e+02,  1.45195e+01,
       -3.02162e+02, -4.42757e+02,  7.13368e+01, -2.82183e+02,
        4.41342e-02,  4.25457e-02,  4.19405e-02])

In [30]:
model.input

<KerasTensor: shape=(None, 60, 31) dtype=float64 (created by layer 'bidirectional_6_input')>